In [84]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, svm, preprocessing, metrics
from sklearn.model_selection import train_test_split

## 建Label dict
- key是FileID
- value是Label
- 方便最後塞Label用

In [124]:
df_train = pd.DataFrame(pd.read_csv('training-set.csv',header=None))
df_train.columns = ['FileID','Label']

dic = df_train.set_index('FileID').to_dict()

dic = dic['Label']

## 讀檔
1. 把之前merge好的大csv讀進來
2. Unnamed: 0欄位是沒用的，先刪掉
3. 增加一個count欄位，方便算比數（應該有更好的寫法）
4. 根據FileID group 幾個欄位出來
    - 'Cnum':每個FileID有對應到的不同的CustomerID
    - 'Pnum':每個FileID有對應到的不同的ProductID
    - 'df_guby_t_max':每個FileID出現的最後時間
    - 'df_guby_t_min':每個FileID第一次出現的時間
    - 'times':每個FileID出現的次數
    - 'time_diff'= df_guby_t_max - df_guby_t_min存在多長的時間
5. 把'df_guby_t_max'及'df_guby_t_min'兩個欄位刪掉（比較沒有意義）
6. 用上面建立好的dict把Label併進來
7. 把Label != 3的歸類為X(train data);Label == 3 的歸類為y(test data)
8. 用train_test_split把X分成訓練跟驗證資料

In [2]:
df_log = pd.DataFrame(pd.read_csv('total_log.csv'))

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_log.head()

,Unnamed: 0,FileID,CustomerID,QueryTS,ProductID,Dtime,Label
0,0,dfccd8e23f0b03ec4db7a9a745ad7399,64573086fcfee53e8fa6d8f1f44101bb,1488326402,c105a0,2017-03-01 00:00:02,3
1,1,29962f9102502ee76bb3cddb8cf8c9ad,597df153a25e8e548cbe35f6e4da9cbb,1488326403,c105a0,2017-03-01 00:00:03,3
2,2,fa3fa642378b9d98c0e2770611b92eb3,1db24b8cc0a7ce59634629e7f292520f,1488326405,c105a0,2017-03-01 00:00:05,3
3,3,48f90ea1af2c04677c6890db392ca088,5ca6dd0b8863069ab1efe014de82cecf,1488326411,c76d58,2017-03-01 00:00:11,3
4,4,d0ff310c25e3bd7ae919d34dc38fb0de,5ca6dd0b8863069ab1efe014de82cecf,1488326411,c76d58,2017-03-01 00:00:11,3


In [8]:
df_log = df_log.drop('Unnamed: 0',axis = 1)

In [27]:
df_log['count'] = 1

In [28]:
df_guby_t_max = df_log.groupby('FileID')['QueryTS'].idxmax()

In [30]:
df_guby_t_min = df_log.groupby('FileID')['QueryTS'].idxmin()

In [11]:
df_guby = df_log.groupby('FileID').CustomerID.nunique()

In [18]:
df_gubyp = df_log.groupby('FileID')['QueryTS']

In [32]:
df_gubycoun = df_log.groupby('FileID')['count'].sum()

In [23]:
gu = pd.DataFrame(df_guby)

In [34]:
train_data = pd.concat([df_guby, df_gubyp, df_guby_t_max, df_guby_t_min, df_gubycoun], axis=1)

In [37]:
train_data.columns = ['Cnum','Pnum','df_guby_t_max','df_guby_t_min','times']

In [38]:
train_data['time_diff'] = train_data['df_guby_t_max'] - train_data['df_guby_t_min']

In [43]:
train_data = train_data.drop(['df_guby_t_max', 'df_guby_t_min'],axis = 1)

In [69]:
train_data['Label'] = 3
for i in range(len(train_data)):
    try:
        train_data['Label'][i] = dic[train_data.index[i]]
    except:
        train_data['Label'][i] = 3

In [72]:
X = train_data[train_data['Label'] != 3]
y = train_data[train_data['Label'] == 3]

In [76]:
X = X.drop('Label',axis = 1)

In [82]:
df_train = df_train.set_index('FileID')

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X, df_train, random_state = 7, test_size = 0.25)

In [ ]:
# x = train_data.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(x_scaled)

## GBM
- 裝XGB的時候碰到了問題，所以先用GBM做了
- 做出來的預測是0,1不是機率（有點大的問題）
- train的時候沒有做很多的校調，一次就傳上去了（結果只有0.5哈哈哈哈（別打我

In [94]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [90]:
clf = GradientBoostingClassifier(n_estimators=500,max_depth=10,min_samples_split=5,min_samples_leaf=3)

In [91]:
clf.fit(X_train,y_train)

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=3, min_samples_split=5,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [92]:
pred_y = clf.predict(X_test)

In [95]:
accuracy = accuracy_score(y_test, pred_y )
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.99%


In [101]:
y_pred = clf.predict(y)

In [100]:
y = y.drop('Label',axis = 1)

In [105]:
y['Probability'] = y_pred

In [112]:
y_final = y.drop(['Cnum','Pnum','times','time_diff'],axis = 1)

In [115]:
y_final['FileID'] = y_final.index

In [119]:
df_y = y_final.reset_index(drop=True)

In [121]:
df_y = df_y[['FileID', 'Probability']]

In [123]:
df_y.to_csv('tbrain_v2.csv', sep=',', encoding='utf-8')